In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [16]:
from master import *
from models import *
from torchsummary import summary

In [17]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
ROOT_DIR = '/beegfs/bva212/fma_small_cqt/'
PATH_FILE = 'checksums'
BATCH_SIZE = 32
VAL_SPLIT = 0.2
TRANSFORM_PROB  = 0.5
SR = 22050
MODEL_SAVE_PATH = '/beegfs/sc6957/audionet_jigsaw_model.pth'
MODE = 'jigsaw'
MODEL_ARCHITECTURE = 'snet2'  #['l3net','snet','anet']
SEPARABLE = False
REDUCE_TWO_CLASS = True #Applies to jigsaw (6-class to 2-class)
NUM_SECONDS=10

cuda:0


In [18]:
start = time.time()
MODEL = load_model(DEVICE, MODEL_ARCHITECTURE, MODE, SEPARABLE)
master = Master(DEVICE, ROOT_DIR, PATH_FILE, SR, BATCH_SIZE, VAL_SPLIT, TRANSFORM_PROB, REDUCE_TWO_CLASS, MODE, MODEL, NUM_SECONDS)
print(f'time for instantiating audio net object - {time.time() - start}')

Instantiating Model snet2_jigsaw
MODE: jigsaw
Getting Train & Validation Datasets
Excluding these 3 files - ['098/098567.mp3_cqt.npy', '098/098565.mp3_cqt.npy', '098/098569.mp3_cqt.npy']
There are a total of 7836 music files in the root directory
# Training samples: 6268 	 # Validation samples: 1567
	 --Done
Creating Train & Validation Dataloaders
Length of Train dataloader: 196 	 Validation dataloader: 49
	 --Done
	 --Done
Init actions done
time for instantiating audio net object - 1.904123067855835


In [19]:
master.model

DataParallel(
  (module): snet2_jigsaw(
    (conv_layers): Sequential(
      (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(2, 2), padding=(5, 5))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(5, 5))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(5, 5))
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
      (9): AvgPool2d(kernel_size=3, stride=3, padding=0)
      (10): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(4, 4))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): Conv2d(128, 256, kernel_size=(4, 4), stride=(1, 1), padding=(4, 

In [20]:
master.input_size

torch.Size([3, 84, 135])

In [21]:
summary(master.model,input_size=master.input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 45, 71]             416
       BatchNorm2d-2           [-1, 16, 45, 71]              32
              ReLU-3           [-1, 16, 45, 71]               0
            Conv2d-4           [-1, 32, 26, 39]          12,832
       BatchNorm2d-5           [-1, 32, 26, 39]              64
              ReLU-6           [-1, 32, 26, 39]               0
            Conv2d-7           [-1, 64, 32, 45]          51,264
       BatchNorm2d-8           [-1, 64, 32, 45]             128
              ReLU-9           [-1, 64, 32, 45]               0
        AvgPool2d-10           [-1, 64, 10, 15]               0
           Conv2d-11          [-1, 128, 15, 20]         131,200
      BatchNorm2d-12          [-1, 128, 15, 20]             256
             ReLU-13          [-1, 128, 15, 20]               0
           Conv2d-14          [-1, 256,

In [ ]:
master.train(num_epochs= 70, learning_rate=1e-5, print_every=4, verbose=True, save=True, checkpoint_every=5)

Instantiating Optimzer, Loss Criterion, Scheduler
	 --Done
Training started
Epoch 1/70
----------
Phase: train   Iteration 196/196
train Loss: 0.7062 Acc: 0.5137
Phase: valid   Iteration 49/49
valid Loss: 0.7301 Acc: 0.5163

Successfully checkpointed model after 0 epochs
Epoch 5/70id   Iteration 49/4996
----------
Phase: train   Iteration 196/196
train Loss: 0.6865 Acc: 0.5310
Phase: valid   Iteration 49/49
valid Loss: 0.6864 Acc: 0.5533

Successfully checkpointed model after 5 epochs
Epoch 9/70id   Iteration 49/4996
----------
Phase: train   Iteration 196/196
train Loss: 0.6663 Acc: 0.5654
Phase: valid   Iteration 49/49
valid Loss: 0.6592 Acc: 0.5801

Successfully checkpointed model after 10 epochs
Epoch 13/70d   Iteration 49/4996
----------
Phase: train   Iteration 196/196
train Loss: 0.6546 Acc: 0.5854
Phase: valid   Iteration 49/49
valid Loss: 0.6493 Acc: 0.5788

Successfully checkpointed model after 15 epochs
Epoch 17/70
----------
Phase: train   Iteration 196/196
train Loss: 0.64

In [ ]:
start = time.time()
batch = next(iter(master.dataloaders['train']))
print(f'time for fetching one batch - {time.time() - start}')

In [79]:
batch[0].shape,batch[1].shape

(torch.Size([32, 3, 84, 135]), torch.Size([32]))